In [1]:
from pyspark.sql import SQLContext
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from flask import Flask
import io
from flask import Response

In [2]:
sqlContext = SQLContext(sc)
c_total_2017 = sqlContext.read.format("org.apache.spark.sql.cassandra").options(table="c_total_2017", keyspace="dgt").load()
c_total_2016 = sqlContext.read.format("org.apache.spark.sql.cassandra").options(table="c_total_2016", keyspace="dgt").load()
c_total_2015 = sqlContext.read.format("org.apache.spark.sql.cassandra").options(table="c_total_2015", keyspace="dgt").load()

In [3]:
p_total_2017 = sqlContext.read.format("org.apache.spark.sql.cassandra").options(table="p_total_2017", keyspace="dgt").load()
p_total_2016 = sqlContext.read.format("org.apache.spark.sql.cassandra").options(table="p_total_2016", keyspace="dgt").load()
p_total_2015 = sqlContext.read.format("org.apache.spark.sql.cassandra").options(table="p_total_2015", keyspace="dgt").load()

In [4]:
t_total_2017 = sqlContext.read.format("org.apache.spark.sql.cassandra").options(table="t_total_2017", keyspace="dgt").load()
t_total_2016 = sqlContext.read.format("org.apache.spark.sql.cassandra").options(table="t_total_2016", keyspace="dgt").load()
t_total_2015 = sqlContext.read.format("org.apache.spark.sql.cassandra").options(table="t_total_2015", keyspace="dgt").load()

In [14]:
df_c_total_2017 = c_total_2017.select('comunidad_autonoma', 'fallecidos').where("comunidad_autonoma NOT LIKE 'Total'").toPandas()
df_c_total_2016 = c_total_2016.select('comunidades_autonomas', 'fallecidos').where("comunidades_autonomas NOT LIKE 'Total'").toPandas()
df_c_total_2015 = c_total_2015.select('comunidades_autonomas', 'fallecidos').where("comunidades_autonomas NOT LIKE 'Total'").toPandas()

In [15]:
p_total_2017 = p_total_2017.withColumnRenamed('a_mortales', 'a_mortales_2017')
p_total_2016 = p_total_2016.withColumnRenamed('a_mortales', 'a_mortales_2016')
p_total_2015 = p_total_2015.withColumnRenamed('a_mortales', 'a_mortales_2015')
join_total = p_total_2017.join(p_total_2016, 'provincia').join(p_total_2015, 'provincia')

In [16]:
def gr_c_total_2017():
    plt.rcParams["figure.figsize"] = (20,10)
    fig = Figure()
    axis = fig.add_subplot(1, 1, 1)
    df_c_total_2017.plot(ax=axis, kind='barh', x='comunidad_autonoma', y='fallecidos', color='red')
    return fig

def gr_c_total_2016():
    plt.rcParams["figure.figsize"] = (20,10)
    fig = Figure()
    axis = fig.add_subplot(1, 1, 1)
    df_c_total_2016.plot(ax=axis, kind='barh', x='comunidades_autonomas', y='fallecidos', color='red')
    return fig

def gr_c_total_2015():
    plt.rcParams["figure.figsize"] = (20,10)
    fig = Figure()
    axis = fig.add_subplot(1, 1, 1)
    df_c_total_2015.plot(ax=axis, kind='barh', x='comunidades_autonomas', y='fallecidos', color='red')
    return fig

In [17]:
def df_buscar(provincia):
    df_busqueda = join_total.select("provincia", "a_mortales_2017", "a_mortales_2016", "a_mortales_2015").where('provincia like "{}"'.format(provincia)).toPandas()
    return df_busqueda

In [18]:
def grafica_busqueda(provincia):
    plt.rcParams["figure.figsize"] = (10,10)
    fig = Figure()
    axis = fig.add_subplot(1, 1, 1)
    df_busqueda = df_buscar(provincia)
    df_busqueda.plot(ax=axis, kind='bar', x='provincia', y=['a_mortales_2017', 'a_mortales_2016', 'a_mortales_2015'], color=['red', 'darkred', 'firebrick'])
    return fig

In [19]:
df_t_total_2017 = t_total_2017.select('tipo_accidente', 'fallecidos').where('tipo_accidente NOT LIKE "Total"').toPandas()
df_t_total_2016 = t_total_2016.select('tipo_accidente', 'fallecidos').where('tipo_accidente NOT LIKE "Total"').toPandas()
df_t_total_2015 = t_total_2015.select('tipo_accidente', 'fallecidos').where('tipo_accidente NOT LIKE "Total"').toPandas()

In [20]:
def gr_t_total_2017():
    plt.rcParams["figure.figsize"] = (20,10)
    fig = Figure()
    axis = fig.add_subplot(1, 1, 1)
    df_t_total_2017.groupby('tipo_accidente')['fallecidos'].sum().plot(ax=axis, kind='barh', color='red')
    return fig

def gr_t_total_2016():
    plt.rcParams["figure.figsize"] = (20,10)
    fig = Figure()
    axis = fig.add_subplot(1, 1, 1)
    df_t_total_2016.groupby('tipo_accidente')['fallecidos'].sum().plot(ax=axis, kind='barh', color='red')
    return fig

def gr_t_total_2015():
    plt.rcParams["figure.figsize"] = (20,10)
    fig = Figure()
    axis = fig.add_subplot(1, 1, 1)
    df_t_total_2015.groupby('tipo_accidente')['fallecidos'].sum().plot(ax=axis, kind='barh', color='red')
    return fig

In [21]:
df_p_total_2017 = p_total_2017.select('provincia', 'fallecidos').where('provincia NOT LIKE "Total"').toPandas()
df_p_total_2016 = p_total_2016.select('provincia', 'fallecidos').where('provincia NOT LIKE "Total"').toPandas()
df_p_total_2015 = p_total_2015.select('provincia', 'fallecidos').where('provincia NOT LIKE "Total"').toPandas()

In [22]:
def gr_p_total_2017():
    plt.rcParams["figure.figsize"] = (10,10)
    fig = Figure()
    axis = fig.add_subplot(1, 1, 1)
    df_p_total_2017.plot(ax=axis, kind='bar', x='provincia', y='fallecidos', color='red')
    return fig

def gr_p_total_2016():
    plt.rcParams["figure.figsize"] = (10,10)
    fig = Figure()
    axis = fig.add_subplot(1, 1, 1)
    df_p_total_2016.plot(ax=axis, kind='bar', x='provincia', y='fallecidos', color='red')
    return fig

def gr_p_total_2015():
    plt.rcParams["figure.figsize"] = (10,10)
    fig = Figure()
    axis = fig.add_subplot(1, 1, 1)
    df_p_total_2015.plot(ax=axis, kind='bar', x='provincia', y='fallecidos', color='red')
    return fig

In [24]:
app = Flask(__name__)
@app.route("/c_total_2017.png")
def c_total_2017():
    fig = gr_c_total_2017()
    output = io.BytesIO()
    FigureCanvas(fig).print_png(output)
    return Response(output.getvalue(), mimetype='image/png')

@app.route("/c_total_2016.png")
def c_total_2016():
    fig = gr_c_total_2016()
    output = io.BytesIO()
    FigureCanvas(fig).print_png(output)
    return Response(output.getvalue(), mimetype='image/png')

@app.route("/c_total_2015.png")
def c_total_2015():
    fig = gr_c_total_2015()
    output = io.BytesIO()
    FigureCanvas(fig).print_png(output)
    return Response(output.getvalue(), mimetype='image/png')

@app.route("/p_total_2017.png")
def p_total_2017():
    fig = gr_p_total_2017()
    output = io.BytesIO()
    FigureCanvas(fig).print_png(output)
    return Response(output.getvalue(), mimetype='image/png')

@app.route("/p_total_2016.png")
def p_total_2016():
    fig = gr_p_total_2016()
    output = io.BytesIO()
    FigureCanvas(fig).print_png(output)
    return Response(output.getvalue(), mimetype='image/png')

@app.route("/p_total_2015.png")
def p_total_2015():
    fig = gr_p_total_2015()
    output = io.BytesIO()
    FigureCanvas(fig).print_png(output)
    return Response(output.getvalue(), mimetype='image/png')

@app.route("/t_total_2017.png")
def t_total_2017():
    fig = gr_t_total_2017()
    output = io.BytesIO()
    FigureCanvas(fig).print_png(output)
    return Response(output.getvalue(), mimetype='image/png')

@app.route("/t_total_2016.png")
def t_total_2016():
    fig = gr_t_total_2016()
    output = io.BytesIO()
    FigureCanvas(fig).print_png(output)
    return Response(output.getvalue(), mimetype='image/png')

@app.route("/t_total_2015.png")
def t_total_2015():
    fig = gr_t_total_2015()
    output = io.BytesIO()
    FigureCanvas(fig).print_png(output)
    return Response(output.getvalue(), mimetype='image/png')

@app.route("/buscar/<provincia_flask>.png")
def busqueda(provincia_flask):
    fig = grafica_busqueda(provincia_flask)
    output = io.BytesIO()
    FigureCanvas(fig).print_png(output)
    return Response(output.getvalue(), mimetype='image/png')
if __name__ == "__main__":
    app.run("192.168.2.2")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.2.2:5000/ (Press CTRL+C to quit)
192.168.2.2 - - [16/Jun/2019 12:15:43] "GET /c_total_2017.png HTTP/1.1" 200 -
192.168.2.2 - - [16/Jun/2019 12:15:48] "GET /c_total_2016.png HTTP/1.1" 200 -
192.168.2.2 - - [16/Jun/2019 12:15:55] "GET /c_total_2015.png HTTP/1.1" 200 -
192.168.2.2 - - [16/Jun/2019 12:16:05] "GET /c_total_2017.png HTTP/1.1" 200 -
192.168.2.2 - - [16/Jun/2019 12:19:05] "GET /c_total_2016.png HTTP/1.1" 200 -
192.168.2.2 - - [16/Jun/2019 12:20:44] "GET /c_total_2015.png HTTP/1.1" 200 -
192.168.1.5 - - [16/Jun/2019 12:22:52] "GET /c_total_2017.png HTTP/1.1" 200 -
192.168.1.5 - - [16/Jun/2019 12:22:54] "GET /c_total_2016.png HTTP/1.1" 200 -
192.168.1.5 - - [16/Jun/2019 12:22:55] "GET /c_total_2015.png HTTP/1.1" 200 -
192.168.1.5 - - [16/Jun/2019 12:22:55] "GET /c_total_2017.png HTTP/1.1" 200 -
192.168.1.5 - - [16/Jun/2019 12:23:03] "GET /c_total_2017.png HTTP/1.1" 200 -
192.168.1.5 - - [16/Jun/2019 12:23:10] "GET /c_total_2017.png HTTP/1.1" 200 -
19